# Traders
This notebook shows how to create your own trader. There are two ways to do so:

1. Extend the `FlexTarder`: this approach is mostly suitable if you just require some minor changes to the default behavior, like creating different order types.
2. Implement the `Trader` from scratch: this approach provides full flexibility on what the policy should do.

In [1]:
%use roboquant(3.0.0-SNAPSHOT)
Welcome()

roboquant 3.0.0-SNAPSHOT 
 build: 2025-05-06T09:22:02Z 
 home: /Users/peter/.roboquant 
 os: Mac OS X 15.4.1 
 jvm: OpenJDK 64-Bit Server VM 17.0.15 
 memory: 3256 MB 
 cpu cores: 8

# Implementing the `Trader` interface

The following cell shows a very minimalistic implementation of a Trader in which BUY ratings are generating buy market orders and SELL ratings are generating sell market orders if there is an open position.

NOTE: In general it is important that a Trader is very robust, especially if you intent to use it also for live trading. It means that all kind of corner cases need to be catered for. A simple implementation as the one below isn't sufficient. Have a look at the `FlexTrader` in the roboquant GitHub repo to see a more complete implementation.

In [2]:
class MyTrader : Trader {

    // This is the only method that is required to implement and transforms incoming signals into orders
    override public fun createOrders(signals: List<Signal>, account: Account, event: Event): List<Order> {

        val orders = mutableListOf<Order>()
        
        for (signal in signals) {
            val asset = signal.asset
            val price = event.getPrice(asset, "CLOSE") ?: continue
            val positionSize = account.positionSize(asset)

            if (signal.rating > 0)
                orders.add(Order(asset, Size(10), price))
            else if (signal.rating < 0 && positionSize > 0)
                orders.add(Order(asset, -positionSize, price))
        }
        
        return orders
    }
    
}

In [3]:
val feed = AvroFeed.sp25()
val strategy = EMACrossover()
val trader = MyTrader()
run(feed, strategy, trader=trader)

last update  : 2024-12-31T21:00:00Z
cash         : USD 984,792.67
buying Power : USD 984,792.67
equity       : USD 1,025,469.71
positions    : 0@WMT, 20@NVDA, 0@JPM, 0@MA, 0@BRK.B, 0@META, 10@ABBV, 10@V, 0@CVX, 10@AVGO, 0@UNH, 0@GOOGL, 0@GOOG, 0@MSFT, 0@JNJ, 0@TSLA, 10@AMZN, 10@AAPL, 0@NFLX, 20@COST, 0@PG, 0@XOM, 10@LLY, 0@MRK, 0@HD
open orders  : -10@AMZN, -20@NVDA